<a href="https://colab.research.google.com/github/teruelda/prices_market/blob/main/Datamarket_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqldf
import pandas as pd
import sqldf 
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Lectura de datos, asignación de datos 'datetime', filtro de dataframe para registros 2022 y configuración para que se muestre por completo el campo 'url':
data = "/content/drive/MyDrive/Colab Notebooks/DATAMARKET/Copia de datamarket_productos_de_supermercados.csv"
df=pd.read_csv(data, parse_dates=['insert_date'])
filter = df["insert_date"]>"2021-12-31"
df_filter = df[filter]
pd.set_option('display.max_colwidth', None)
df_filter

,id,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
4540977,20678234,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,mercadona-es,fruta_y_verdura_verdura,Tomates kumato,Bandeja,1.99,1.990,kg,2022-01-01,3c718e914606384450d99a262132df08
4540978,20674817,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,mercadona-es,carne_aves_y_pollo,Tacos de pavo,Bandeja,3.65,6.090,kg,2022-01-01,17a2d0996687086917e3340e47b25a32
4540979,20674633,https://tienda.mercadona.es/product/28811/sidra-gaitero-botella,mercadona-es,bodega_sidra_y_cava,Sidra El Gaitero,Botella,1.85,2.467,l,2022-01-01,2922d0b2965f12742a821958ac7d0b11
4540980,20672497,https://tienda.mercadona.es/product/67147/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.60,0.910,l,2022-01-01,bbd065eaaf47f1ee4b9c03b5c64f8ebb
4540981,20672499,https://tienda.mercadona.es/product/67146/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.24,1.350,l,2022-01-01,3a974af2f23c43d9739d4d0551ccce93
...,...,...,...,...,...,...,...,...,...,...,...
5995595,22673267,https://tienda.mercadona.es/product/34922/almendra-cubitos-hacendado-paquete,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Almendra cubitos Hacendado,Paquete,2.15,17.200,kg,2022-04-10,d0cb5f4535e1e38b3962fa25dcf90e55
5995596,22673269,https://tienda.mercadona.es/product/22241/sirope-fresa-hacendado-bote,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Sirope de fresa Hacendado,Bote,1.30,4.334,kg,2022-04-10,8af2b3f95b7967e14ee3879797d7bdcb
5995597,22673271,https://tienda.mercadona.es/product/32357/colorante-alimentario-4-colores-gel-hacendado-caja,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Colorante alimentario 4 colores en gel Hacendado,Caja,2.25,5.625,100g,2022-04-10,0c216cede11cd58bf299b8a4e07b7fe3
5995598,22673409,https://tienda.mercadona.es/product/63627/pizza-jamon-queso-familiar-hacendado,mercadona-es,pizzas_y_platos_preparados_pizzas,Pizza jamón y queso familiar Hacendado,NaN,3.10,5.167,kg,2022-04-10,07305d76125da83d0cd4db4ab5648605


In [ ]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454623 entries, 4540977 to 5995599
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   id               1454623 non-null  int64         
 1   url              1454623 non-null  object        
 2   supermarket      1454623 non-null  object        
 3   category         1454623 non-null  object        
 4   name             1454623 non-null  object        
 5   description      538016 non-null   object        
 6   price            1454623 non-null  float64       
 7   reference_price  1454623 non-null  float64       
 8   reference_unit   1453277 non-null  object        
 9   insert_date      1454623 non-null  datetime64[ns]
 10  product_id       1454623 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 133.2+ MB


In [ ]:
#Se realizan una serie de comprobaciones:
last_date = df_filter.iloc[-1,9] # última fecha de registro del dataset
window_date_30_days = last_date - pd.Timedelta("30 days") # fecha con 30 días de antelación a la última fecha de registro
n_days = len(df_filter['insert_date'].unique()) # número de días diferentes registrados en el dataset
n_categories = len(df_filter['category'].unique()) # número de categorías diferentes en el dataset
n_products = len(df_filter['url'].unique()) # número de productos diferentes en el dataset, entendiendo que cada url pertenece a un producto
n_supermarket = len(df_filter['supermarket'].unique()) #número de supermercados diferentes 

print("last_date: " + str(last_date) + "\n" 
      + "window_date_30_days: " + str(window_date_30_days) + "\n" 
      + "n_days: " + str(n_days) + "\n" 
      + "n_categories: " + str(n_categories) + "\n" 
      + "n_products: " + str(n_products) + "\n"
      + "n_supermarket: " + str(n_supermarket))

last_date: 2022-04-10 00:00:00
window_date_30_days: 2022-03-11 00:00:00
n_days: 100
n_categories: 815
n_products: 23303
n_supermarket: 3


In [ ]:
# FUNCIÓN DE CODIFICACIÓN DE CAMPOS
# El objeto de esta función es la de asignar una codificación numérica para cada campo del dataset que se desee codificar:
# Parámetros de entrada:
#   - dataset: debe ser un dataframe
#   - campo: deber ser texto, indicando el campo del dataset que se desea codificar

def CODE(dataset, campo):
  valores = dataset[campo].unique()
  return pd.DataFrame({campo:valores}).reset_index().rename(columns={"index":"ID"})

In [ ]:
# Con la función "CODE" se asigna coificación a los siguientes campos: "url", "category", "supermarket" 

ID_product = CODE(df_filter,"url")
ID_category = CODE(df_filter, "category")
ID_supermarket = CODE(df_filter, "supermarket")

# Se realiza la comprobación de que la longitud de cada una de estas nuevas tablas debe coincidir con los valores únicos detectados en el primer dataset

print(ID_product.shape[0])
print(ID_category.shape[0])
print(ID_supermarket.shape[0])


23303
815
3


In [ ]:
df_pru = df_filter.copy()

In [ ]:
# Una vez codificados los campos deseados, se aplica la codificación en el dataset mediante la función "APPLY_CODE"

def APPLY_CODE(df_entry, df_campo, key_campo, value_campo, ID_campo):

  key_dic = df_campo[key_campo].tolist()
  value_dic = df_campo[value_campo].tolist()
  dic = dict(zip(key_dic, value_dic))

  new_df = df_entry.copy()
  new_df[ID_campo] = new_df[key_campo].map(dic)

  return new_df


In [ ]:
df_filter_with_ID = APPLY_CODE(df_pru, ID_product, "url", "ID", "ID_product") 
df_filter_with_ID = APPLY_CODE(df_filter_with_ID, ID_category, "category", "ID", "ID_category")
df_filter_with_ID = APPLY_CODE(df_filter_with_ID, ID_supermarket, "supermarket", "ID", "ID_supermarket")
df_filter_with_ID

,id,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,ID_product,ID_category,ID_supermarket
4540977,20678234,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,mercadona-es,fruta_y_verdura_verdura,Tomates kumato,Bandeja,1.99,1.990,kg,2022-01-01,3c718e914606384450d99a262132df08,0,0,0
4540978,20674817,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,mercadona-es,carne_aves_y_pollo,Tacos de pavo,Bandeja,3.65,6.090,kg,2022-01-01,17a2d0996687086917e3340e47b25a32,1,1,0
4540979,20674633,https://tienda.mercadona.es/product/28811/sidra-gaitero-botella,mercadona-es,bodega_sidra_y_cava,Sidra El Gaitero,Botella,1.85,2.467,l,2022-01-01,2922d0b2965f12742a821958ac7d0b11,2,2,0
4540980,20672497,https://tienda.mercadona.es/product/67147/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.60,0.910,l,2022-01-01,bbd065eaaf47f1ee4b9c03b5c64f8ebb,3,3,0
4540981,20672499,https://tienda.mercadona.es/product/67146/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.24,1.350,l,2022-01-01,3a974af2f23c43d9739d4d0551ccce93,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995595,22673267,https://tienda.mercadona.es/product/34922/almendra-cubitos-hacendado-paquete,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Almendra cubitos Hacendado,Paquete,2.15,17.200,kg,2022-04-10,d0cb5f4535e1e38b3962fa25dcf90e55,2981,207,0
5995596,22673269,https://tienda.mercadona.es/product/22241/sirope-fresa-hacendado-bote,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Sirope de fresa Hacendado,Bote,1.30,4.334,kg,2022-04-10,8af2b3f95b7967e14ee3879797d7bdcb,18874,207,0
5995597,22673271,https://tienda.mercadona.es/product/32357/colorante-alimentario-4-colores-gel-hacendado-caja,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Colorante alimentario 4 colores en gel Hacendado,Caja,2.25,5.625,100g,2022-04-10,0c216cede11cd58bf299b8a4e07b7fe3,22648,207,0
5995598,22673409,https://tienda.mercadona.es/product/63627/pizza-jamon-queso-familiar-hacendado,mercadona-es,pizzas_y_platos_preparados_pizzas,Pizza jamón y queso familiar Hacendado,NaN,3.10,5.167,kg,2022-04-10,07305d76125da83d0cd4db4ab5648605,9139,240,0


In [ ]:
# Se debe garantizar que un producto está asignado únicamente a una categoría:
# Al realizar una agrupación por producto y categoría, la tabla resultante arroja más registros de los 23303 productos diferentes que existen, lo que lleva a concluir
# que existen productos que tienen asignados más de una categoría. Para resolver esta incidencia se sigue el siguiente criterio de selección de categoria para cada producto:
#    - Categoría que más veces se repite
#    - En caso que el número de registros de un producto con diferentes categorías coincida, se escogerá la categoría por orden alfabético.

df_product_category = df_filter_with_ID.groupby(["ID_product", "category"]
                          )["url"].count().reset_index().rename(columns={"url":"n_categ"}).sort_values(
                          ["ID_product","n_categ","category"], ascending=[True,False,True]).drop_duplicates(subset="ID_product", keep="first")
df_product_category

,ID_product,category,n_categ
0,0,fruta_y_verdura_verdura,100
1,1,carne_aves_y_pollo,100
2,2,bodega_sidra_y_cava,100
3,3,bodega_cerveza,100
4,4,bodega_cerveza,100
...,...,...,...
25099,23298,parafarmacia_higiene_bucal_colutorio,1
25100,23299,la_despensa_yogures_y_postres_gelatina,1
25101,23300,bebidas_batidos_y_horchata,1
25102,23301,bebidas_batidos_y_horchata,1


In [ ]:
#Asignación de categoría a cada producto:
product_category = sqldf.run("SELECT t1.ID_product, t1.category, t2.ID as ID_category FROM df_product_category as t1 LEFT JOIN ID_category as t2 ON t1.category = t2.category")
product_category

,ID_product,category,ID_category
0,0,fruta_y_verdura_verdura,0
1,1,carne_aves_y_pollo,1
2,2,bodega_sidra_y_cava,2
3,3,bodega_cerveza,3
4,4,bodega_cerveza,3
...,...,...,...
23298,23298,parafarmacia_higiene_bucal_colutorio,656
23299,23299,la_despensa_yogures_y_postres_gelatina,708
23300,23300,bebidas_batidos_y_horchata,151
23301,23301,bebidas_batidos_y_horchata,151


In [ ]:
len(product_category["ID_category"].unique()) #se observa que solo hay 806 registros de categoría en lugar de 815. Esto se explica porque al tener algunos productos con diferentes
#categorias inicialmente, al considerar 1 producto = 1 categoría, se han eliminado categorías que estaban mal escritas o asignadas incorrectamente.

806

In [ ]:
#Igual ocurre para el campo "name", se observa que el problema radica en que hay diferentes nombres para un mismo producto. Se procede a la limpieza con los mismos criterios que 
#la limpieza del campo "category": se queda el "name" con mayor número de repeticiones y en caso de empate, el primero por orden alfabético.
name_table = df_filter_with_ID.groupby(["ID_product", "name"]
                                       )["url"].count().reset_index().rename(columns={"url":"Num"}).sort_values(["ID_product","Num","name"], ascending=[True,False,True])
name_table = name_table.drop_duplicates(subset="ID_product", keep="first")
name_table

,ID_product,name,Num
0,0,Tomates kumato,100
1,1,Tacos de pavo,100
2,2,Sidra El Gaitero,100
3,3,Cerveza Clásica Steinburg,100
4,4,Cerveza Clásica Steinburg,100
...,...,...,...
23760,23298,"Colutorio diario 0,05% con sabor a fresa sin alcohol Flúor Lacer 500 ml.",1
23761,23299,Gelatina sabor arándanos antiox sin azúcar añadido Royal pack de 4 unidades de 100 g.,1
23762,23300,DIA horchata envase 1 lt,1
23763,23301,CHUFI horchata botella 1 lt,1


In [ ]:
product_name = sqldf.run("SELECT DISTINCT ID_product, name, description FROM df_filter_with_ID")
product_name # al obtener esta tabla se observa que el número de registros es superior a 23303, lo que indica que bien hay diferentes "name" o "description" asignados a un mismo ID_product

,ID_product,name,description
0,0,Tomates kumato,Bandeja
1,1,Tacos de pavo,Bandeja
2,2,Sidra El Gaitero,Botella
3,3,Cerveza Clásica Steinburg,Pack-12
4,4,Cerveza Clásica Steinburg,Pack-12
...,...,...,...
23760,23298,"Colutorio diario 0,05% con sabor a fresa sin alcohol Flúor Lacer 500 ml.",None
23761,23299,Gelatina sabor arándanos antiox sin azúcar añadido Royal pack de 4 unidades de 100 g.,None
23762,23300,DIA horchata envase 1 lt,None
23763,23301,CHUFI horchata botella 1 lt,None


In [ ]:
#Se procede a implementar en el dataset el nuevo código de category y el name:
df_filter_with_ID = APPLY_CODE(df_filter_with_ID, product_category, "ID_product", "category", "category")
df_filter_with_ID = APPLY_CODE(df_filter_with_ID, ID_category, "category", "ID", "ID_category")
df_filter_with_ID = APPLY_CODE(df_filter_with_ID, name_table, "ID_product", "name", "correct_name")

In [ ]:
df_filter_with_ID

,id,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,ID_product,ID_category,ID_supermarket,correct_name
4540977,20678234,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,mercadona-es,fruta_y_verdura_verdura,Tomates kumato,Bandeja,1.99,1.990,kg,2022-01-01 00:00:00,3c718e914606384450d99a262132df08,0,0,0,Tomates kumato
4540978,20674817,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,mercadona-es,carne_aves_y_pollo,Tacos de pavo,Bandeja,3.65,6.090,kg,2022-01-01 00:00:00,17a2d0996687086917e3340e47b25a32,1,1,0,Tacos de pavo
4540979,20674633,https://tienda.mercadona.es/product/28811/sidra-gaitero-botella,mercadona-es,bodega_sidra_y_cava,Sidra El Gaitero,Botella,1.85,2.467,l,2022-01-01 00:00:00,2922d0b2965f12742a821958ac7d0b11,2,2,0,Sidra El Gaitero
4540980,20672497,https://tienda.mercadona.es/product/67147/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.60,0.910,l,2022-01-01 00:00:00,bbd065eaaf47f1ee4b9c03b5c64f8ebb,3,3,0,Cerveza Clásica Steinburg
4540981,20672499,https://tienda.mercadona.es/product/67146/cerveza-clasica-steinburg-pack-12,mercadona-es,bodega_cerveza,Cerveza Clásica Steinburg,Pack-12,3.24,1.350,l,2022-01-01 00:00:00,3a974af2f23c43d9739d4d0551ccce93,4,3,0,Cerveza Clásica Steinburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995595,22673267,https://tienda.mercadona.es/product/34922/almendra-cubitos-hacendado-paquete,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Almendra cubitos Hacendado,Paquete,2.15,17.200,kg,2022-04-10 00:00:00,d0cb5f4535e1e38b3962fa25dcf90e55,2981,207,0,Almendra cubitos Hacendado
5995596,22673269,https://tienda.mercadona.es/product/22241/sirope-fresa-hacendado-bote,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Sirope de fresa Hacendado,Bote,1.30,4.334,kg,2022-04-10 00:00:00,8af2b3f95b7967e14ee3879797d7bdcb,18874,207,0,Sirope de fresa Hacendado
5995597,22673271,https://tienda.mercadona.es/product/32357/colorante-alimentario-4-colores-gel-hacendado-caja,mercadona-es,panaderia_y_pasteleria_velas_y_decoracion,Colorante alimentario 4 colores en gel Hacendado,Caja,2.25,5.625,100g,2022-04-10 00:00:00,0c216cede11cd58bf299b8a4e07b7fe3,22648,207,0,Colorante alimentario 4 colores en gel Hacendado
5995598,22673409,https://tienda.mercadona.es/product/63627/pizza-jamon-queso-familiar-hacendado,mercadona-es,pizzas_y_platos_preparados_pizzas,Pizza jamón y queso familiar Hacendado,None,3.10,5.167,kg,2022-04-10 00:00:00,07305d76125da83d0cd4db4ab5648605,9139,240,0,Pizza jamón y queso familiar Hacendado


In [ ]:
len(df_filter_with_ID['correct_name'].unique()) #al salir un valor inferior a 23303 refleja que hay productos que tienen el mismo nombre. En principio esto no afecta dado
# que cada producto está diferenciado con la clave ID_product.

22573

In [ ]:
#CREACION DE LA TABLA PRECIOS

price_table = sqldf.run("SELECT insert_date, ID_product, price, reference_price FROM df_filter_with_ID")
price_table

,insert_date,ID_product,price,reference_price
0,2022-01-01 00:00:00,0,1.99,1.990
1,2022-01-01 00:00:00,1,3.65,6.090
2,2022-01-01 00:00:00,2,1.85,2.467
3,2022-01-01 00:00:00,3,3.60,0.910
4,2022-01-01 00:00:00,4,3.24,1.350
...,...,...,...,...
1454618,2022-04-10 00:00:00,2981,2.15,17.200
1454619,2022-04-10 00:00:00,18874,1.30,4.334
1454620,2022-04-10 00:00:00,22648,2.25,5.625
1454621,2022-04-10 00:00:00,9139,3.10,5.167


In [ ]:
# CREACION TABLA SUPERMARKET

supermarket_table = sqldf.run("SELECT ID_supermarket, supermarket, COUNT(DISTINCT ID_product) AS n_products_market FROM df_filter_with_ID GROUP BY ID_supermarket")
supermarket_table

,ID_supermarket,supermarket,n_products_market
0,0,mercadona-es,6832
1,1,dia-es,8068
2,2,carrefour-es,8403


In [ ]:
# CREACION TABLA CATEGORY

category_table = sqldf.run("SELECT ID_category, category, COUNT(DISTINCT ID_product) as n_product_category FROM df_filter_with_ID GROUP BY ID_category")
category_table

,ID_category,category,n_product_category
0,0,fruta_y_verdura_verdura,106
1,1,carne_aves_y_pollo,63
2,2,bodega_sidra_y_cava,22
3,3,bodega_cerveza,105
4,4,bodega_licores,80
...,...,...,...
801,808,limpieza_y_hogar_papeleria_forralibros,4
802,809,bebidas_alcoholes_sin_alcohol,1
803,810,productos_frescos_charcuteria_y_quesos_al_corte_quesos_barra_y_frescos,9
804,811,perfumeria_e_higiene_cuidado_y_proteccion_corporal_protectores_solares_y_after_sun,24


In [ ]:
# CREACION TABLA PRODUCTS

product_table = sqldf.run("SELECT ID_product, correct_name, description, url, reference_unit, ID_supermarket, ID_category FROM df_filter_with_ID GROUP BY ID_product")
product_table

,ID_product,correct_name,description,url,reference_unit,ID_supermarket,ID_category
0,0,Tomates kumato,Bandeja,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,kg,0,0
1,1,Tacos de pavo,Bandeja,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,kg,0,1
2,2,Sidra El Gaitero,Botella,https://tienda.mercadona.es/product/28811/sidra-gaitero-botella,l,0,2
3,3,Cerveza Clásica Steinburg,Pack-12,https://tienda.mercadona.es/product/67147/cerveza-clasica-steinburg-pack-12,l,0,3
4,4,Cerveza Clásica Steinburg,Pack-12,https://tienda.mercadona.es/product/67146/cerveza-clasica-steinburg-pack-12,l,0,3
...,...,...,...,...,...,...,...
23298,23298,"Colutorio diario 0,05% con sabor a fresa sin alcohol Flúor Lacer 500 ml.",None,https://www.carrefour.es/supermercado/colutorio-diario-005-con-sabor-a-fresa-sin-alcohol-fluor-lacer-500-ml/R-590508004/p,100ml,2,656
23299,23299,Gelatina sabor arándanos antiox sin azúcar añadido Royal pack de 4 unidades de 100 g.,None,https://www.carrefour.es/supermercado/gelatina-sabor-arandanos-antiox-sin-azucar-anadido-royal-pack-de-4-unidades-de-100-g/R-prod1000382/p,kg,2,708
23300,23300,DIA horchata envase 1 lt,None,https://www.dia.es/compra-online/bebidas/batidos-y-horchata/p/26053,l,1,151
23301,23301,CHUFI horchata botella 1 lt,None,https://www.dia.es/compra-online/bebidas/batidos-y-horchata/p/5137,l,1,151


In [ ]:
# CREACION TABLA KPI_PRODUCTS
#Campos: ID_product - Nº_registers - RM_mean_price - YTD_mean_price - Var_mean_price_1m - Porc_Var_mean_price_1m

KPI_products = sqldf.run("SELECT ID_product, COUNT(ID_product) as n_registers FROM df_filter_with_ID GROUP BY ID_product")

KPI_products["RM_mean_price"] = df_filter_with_ID.groupby(["ID_product"])["price"].rolling(2).mean().reset_index().rename(
                                columns={"price":"RM_mean_price"}).fillna(0).drop_duplicates(subset="ID_product", keep="last")["RM_mean_price"].tolist()

MTD_mean = df_filter_with_ID[df_filter_with_ID["insert_date"]>"2022-03-31 00:00:00"].groupby("ID_product")["price"].mean().reset_index().rename(columns={"price":"MTD_mean_price"})

KPI_products["MTD_mean_price"] = sqldf.run(
                                "SELECT t1.ID_product, t2.MTD_mean_price FROM KPI_products as t1 LEFT JOIN MTD_mean as t2 ON t1.ID_product = t2.ID_product")["MTD_mean_price"].tolist()
#prod_RM_mean_fix = prod_RM_mean.drop_duplicates(subset="ID_product", keep="last")
#prod_MTD_mean = df_filter_with_ID[df_filter_with_ID["insert_date"]>"2022-03-31 00:00:00"].groupby("ID_product")["price"].mean()
KPI_products

,ID_product,n_registers,RM_mean_price,MTD_mean_price
0,0,100,1.99,1.99
1,1,100,3.84,3.84
2,2,100,1.85,1.85
3,3,100,3.84,3.84
4,4,100,3.48,3.48
...,...,...,...,...
23298,23298,1,0.00,5.99
23299,23299,1,0.00,1.71
23300,23300,1,0.00,0.99
23301,23301,1,0.00,1.59


In [ ]:
df_sort = df_filter_with_ID.sort_values(["ID_product", "insert_date"])
df_sort["RM_mean"] = df_filter_with_ID.sort_values("insert_date").groupby("ID_product")["price"].rolling(2).mean().reset_index().fillna(0)["price"].tolist()
df_1m = df_sort[(df_sort["insert_date"]==str(last_date)) | (df_sort["insert_date"]==str(window_date_30_days))]
df_1m.groupby("ID_product")["RM_mean"].rolling(2).sum()

ID_product         
0           5552969     NaN
            5987614    3.98
1           5561223     NaN
            5991074    7.68
2           5559689     NaN
                       ... 
23298       5994260     NaN
23299       5994444     NaN
23300       5995119     NaN
23301       5995122     NaN
23302       5995476     NaN
Name: RM_mean, Length: 29070, dtype: float64

In [ ]:
df_1m

,id,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,ID_product,ID_category,ID_supermarket,correct_name,RM_mean
5552969,22089439,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,mercadona-es,fruta_y_verdura_verdura,Tomates kumato,Bandeja,1.99,1.990,kg,2022-03-11 00:00:00,3c718e914606384450d99a262132df08,0,0,0,Tomates kumato,1.99
5987614,22669644,https://tienda.mercadona.es/product/69976/tomates-kumato-bandeja,mercadona-es,fruta_y_verdura_verdura,Tomates kumato,Bandeja,1.99,1.990,kg,2022-04-10 00:00:00,3c718e914606384450d99a262132df08,0,0,0,Tomates kumato,1.99
5561223,22084177,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,mercadona-es,carne_aves_y_pollo,Tacos de pavo,Bandeja,3.84,6.400,kg,2022-03-11 00:00:00,17a2d0996687086917e3340e47b25a32,1,1,0,Tacos de pavo,3.84
5991074,22664373,https://tienda.mercadona.es/product/2792/tacos-pavo-bandeja,mercadona-es,carne_aves_y_pollo,Tacos de pavo,Bandeja,3.84,6.400,kg,2022-04-10 00:00:00,17a2d0996687086917e3340e47b25a32,1,1,0,Tacos de pavo,3.84
5559689,22083884,https://tienda.mercadona.es/product/28811/sidra-gaitero-botella,mercadona-es,bodega_sidra_y_cava,Sidra El Gaitero,Botella,1.85,2.467,l,2022-03-11 00:00:00,2922d0b2965f12742a821958ac7d0b11,2,2,0,Sidra El Gaitero,1.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994260,22679898,https://www.carrefour.es/supermercado/colutorio-diario-005-con-sabor-a-fresa-sin-alcohol-fluor-lacer-500-ml/R-590508004/p,carrefour-es,parafarmacia_higiene_bucal_colutorio,"Colutorio diario 0,05% con sabor a fresa sin alcohol Flúor Lacer 500 ml.",None,5.99,1.200,100ml,2022-04-10 00:00:00,57b75021e3458e351de55b7389fd3e5e,23298,656,2,"Colutorio diario 0,05% con sabor a fresa sin alcohol Flúor Lacer 500 ml.",0.00
5994444,22681576,https://www.carrefour.es/supermercado/gelatina-sabor-arandanos-antiox-sin-azucar-anadido-royal-pack-de-4-unidades-de-100-g/R-prod1000382/p,carrefour-es,la_despensa_yogures_y_postres_gelatina,Gelatina sabor arándanos antiox sin azúcar añadido Royal pack de 4 unidades de 100 g.,None,1.71,4.270,kg,2022-04-10 00:00:00,e6a9903ab1056dbcfa828e6a4e3d23e8,23299,708,2,Gelatina sabor arándanos antiox sin azúcar añadido Royal pack de 4 unidades de 100 g.,0.00
5995119,22671664,https://www.dia.es/compra-online/bebidas/batidos-y-horchata/p/26053,dia-es,bebidas_batidos_y_horchata,DIA horchata envase 1 lt,None,0.99,0.990,l,2022-04-10 00:00:00,743368eb2bdbe33bef69aa6d2878e6a9,23300,151,1,DIA horchata envase 1 lt,0.00
5995122,22671670,https://www.dia.es/compra-online/bebidas/batidos-y-horchata/p/5137,dia-es,bebidas_batidos_y_horchata,CHUFI horchata botella 1 lt,None,1.59,1.590,l,2022-04-10 00:00:00,d9747e913434f9bb29ec4792e20f5f3e,23301,151,1,CHUFI horchata botella 1 lt,0.00


In [ ]:
df_1m.groupby(["ID_product"])["RM_mean"].diff().reset_index().fillna(0).rename(columns={"RM_mean":"Var_mean_1m"})

,index,Var_mean_1m
0,5552969,0.0
1,5987614,0.0
2,5561223,0.0
3,5991074,0.0
4,5559689,0.0
...,...,...
29065,5994260,0.0
29066,5994444,0.0
29067,5995119,0.0
29068,5995122,0.0


In [ ]:
prod_RM_mean[(prod_RM_mean["insert_date"]==str(last_date)) | (prod_RM_mean["insert_date"]==str(window_date_30_days))]

,insert_date,ID_product,level_2,RM_mean_price
1008788,2022-03-11 00:00:00,0,5552969,NaN
1008789,2022-03-11 00:00:00,1,5561223,NaN
1008790,2022-03-11 00:00:00,2,5559689,NaN
1008791,2022-03-11 00:00:00,3,5553866,NaN
1008792,2022-03-11 00:00:00,4,5553867,NaN
...,...,...,...,...
1454618,2022-04-10 00:00:00,23298,5994260,NaN
1454619,2022-04-10 00:00:00,23299,5994444,NaN
1454620,2022-04-10 00:00:00,23300,5995119,NaN
1454621,2022-04-10 00:00:00,23301,5995122,NaN
